In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os

# Add the parent directory to the path so we can import the module
sys.path.append(os.path.abspath(os.path.join('..')))

In [ ]:
from modules.multiple_align import align_multiple_sequences
from modules.needleman_wunsch import needleman_wunsch, print_alignments
from modules.UPGMA import UPGMA, mat_distances
from modules.nw_multiple import needleman_wunsch_multiple

## Needleman-Wunsch algorithm


First and foremost, we implemented the Needleman-Wunsch algorithm for aligning N sequences. Simple test for 2 and 5 sequences are presented below. 


It is possible to use BLOSSUM matrix and to use your own parametres for costs. It is possible to use different gap opening cost and gap extension cost also.

In [ ]:
sequences = ["CHAT", "CAT"]

score, alignment = needleman_wunsch(sequences, True, print_result=True)

In [ ]:
blossum = True
sequences = ["CHAT", "CAT", "HER", "HAT", "HARAT"]

score, alignment = needleman_wunsch(sequences, blossum, print_result=True)

In [ ]:
blossum = True
sequences1 = ["CHAT", "C-AT"]
sequences2 = ["HER--", "HA--T", "HARAT"]

score, alignment = needleman_wunsch_multiple(sequences1, sequences2, blossum, print_result=True)

#### Needleman-Wunsch Evaluation

In [ ]:
from Bio import pairwise2
from Bio.Align import substitution_matrices


In [ ]:
import Levenshtein

def string_similarity_Levenshtein(str1, str2):
    """
    Computes the normalized Levenshtein similarity between two strings.
    
    Parameters:
    ----------
    str1 : str
        First string.
    str2 : str
        Second string.

    Returns:
    -------
    float
        Similarity score between 0 and 1 (1 = identical, 0 = completely different).
    """
    max_length = max(len(str1), len(str2))
    if max_length == 0:
        return 1.0  # Both strings are empty
    return 1 - Levenshtein.distance(str1, str2) / max_length

In [ ]:
def alignment_result_comparison(result1, result2):
    """ 
    Compare two alignment results

    Parameters:
    result1 (score, alignment1, alignment2): Custom Needleman-Wunsch alignment result
    result2 (score, alignment1, alignment2): Biopython Needleman-Wunsch alignment result
    """
    flag = True
    score1, score2 = result1[0], result2[0]
    if score1 != score2: 
        print(f"Scores are different: {score1} != {score2}")
        flag = False

    alignment1, alignment2 = result1[1], result2[1]
    if (len(alignment1) != len(alignment2)): 
        print(f"Lengths of alignments are different: {len(alignment1)} != {len(alignment2)}")
        flag = False
    alignment_diff = string_similarity_Levenshtein(alignment1, alignment2)
    if alignment_diff != 1: 
        print(f"Alignments are different, Levenshtein distance: {alignment_diff}")
        flag = False

    alignment1, alignment2 = result1[2], result2[2]
    if (len(alignment1) != len(alignment2)): 
        print(f"Lengths of alignments are different: {len(alignment1)} != {len(alignment2)}")
        flag = False
    alignment_diff = string_similarity_Levenshtein(alignment1, alignment2)
    if alignment_diff != 1: 
        print(f"Alignments are different, Levenshtein distance: {alignment_diff}")
        flag = False

    if flag: print("Comparison of custom and Biopython Needleman-Wunsch results passed: alignments are the same and score is the same")

In [ ]:
substitution_matrix = substitution_matrices.load("BLOSUM62")
gap_open_penalty = -10
gap_extension_penalty = -2
seq1 = "CHAT"
seq2 = "CAT"

alignments_biopython = pairwise2.align.globalds(seq1, seq2, substitution_matrix, gap_open_penalty, gap_extension_penalty)
print("Biopython results: \n",alignments_biopython)
print(" \nCustom results:")
score, alignment = needleman_wunsch([seq1, seq2], True, print_result=True)

custom_result = (score, alignment[0], alignment[1])
result_biopython = (alignments_biopython[0][2], alignments_biopython[0][0], alignments_biopython[0][1])
alignment_result_comparison(custom_result, result_biopython)

In [ ]:
substitution_matrix = substitution_matrices.load("BLOSUM62")
gap_open_penalty = -10
gap_extension_penalty = -2
seq2 = "MCGNIQLEYAHHGPATQFLWTYIMIGCLKFKGFREQHFYIPGICKDWHFKFLCFYRMIHIPIGPGYITQNTSPAGHYRHSEKAICVMQMFKYICRFRA"
seq1 = "MHGQLEYIAHSPATRFLYTIGCLKFKWFREHHFNIPGECKDWHFKFDCFYRMIHIPIGPAIMYITSPAGHYRHSEMAITVMQMNKVGCRFRDICLYFVES"

alignments_biopython = pairwise2.align.globalds(seq1, seq2, substitution_matrix, gap_open_penalty, gap_extension_penalty)
print("Biopython results:")
print_alignments([alignments_biopython[0][0], alignments_biopython[0][1]])
print("Score:", alignments_biopython[0][2])


result_biopython = (alignments_biopython[0][2], alignments_biopython[0][0], alignments_biopython[0][1])

In [ ]:
substitution_matrix = substitution_matrices.load("BLOSUM62")
gap_open_penalty = -10
gap_extension_penalty = -2
seq1 = "MCGNIQLEYAHHGPATQFLWTYIMIGCLKFKGFREQHFYIPGICKDWHFKFLCFYRMIHIPIGPGYITQNTSPAGHYRHSEKAICVMQMFKYICRFRA"
seq2 = "MHGQLEYIAHSPATRFLYTIGCLKFKWFREHHFNIPGECKDWHFKFDCFYRMIHIPIGPAIMYITSPAGHYRHSEMAITVMQMNKVGCRFRDICLYFVES"

print(" \nCustom results:")
score, alignment = needleman_wunsch([seq1, seq2], True, print_result=False)
print_alignments(alignment)
print("Score: ", score)

custom_result = (score, alignment[0], alignment[1])

In [ ]:
alignment_result_comparison(custom_result, result_biopython)

## UPGMA Algorithm

We used the UPGMA (Unweighted Pair Group Method with Arithmetic Mean) algorithm for clustering. Below are some simple examples demonstrating the functionality of the algorithm.


In [ ]:
from graphviz import Digraph
from IPython.display import display

def visualize_tree(tree, sequences):
        """
        Displays a graphical representation of the tree using Graphviz in Jupyter Notebook.

        Returns:
        -------
        None (renders the graph inline in Jupyter Notebook)
        """
        def add_nodes_edges(tree, graph, node_id=0):
            """ Recursively adds nodes and edges to the Graphviz graph. """
            if tree is not None:
                if len(tree.val) > 1:
                    node_label = ", ".join([sequences[i] for i in tree.val])
                else:
                    node_label = str(sequences[tree.val[0]])
                graph.node(str(node_id), node_label)

                left_id, right_id = node_id * 2 + 1, node_id * 2 + 2
                if tree.left:
                    graph.edge(str(node_id), str(left_id), label="L")
                    add_nodes_edges(tree.left, graph, left_id)
                if tree.right:
                    graph.edge(str(node_id), str(right_id), label="R")
                    add_nodes_edges(tree.right, graph, right_id)

        dot = Digraph(format="png") 
        add_nodes_edges(tree, dot)
        display(dot)

In [ ]:
sequences = ["CHAT", "CAT"]

tree = UPGMA(sequences, True)
visualize_tree(tree, sequences)


In [ ]:
sequences = ["CHAT", "CAT", "HER", "HAT", "HARAT"]

tree = UPGMA(sequences, True)
visualize_tree(tree, sequences)

### UPGMA Evaluation

In [ ]:
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor, DistanceMatrix
from Bio.Phylo import draw_ascii

def upgma_biopython(distance_matrix, sequence_labels):
    """
    Computes UPGMA clustering using Biopython's implementation.

    Parameters:
    ----------
    distance_matrix : list of list of float
        A symmetric matrix containing pairwise distances between sequences.
    sequence_labels : list of str
        List of sequence labels corresponding to the distance matrix.

    Returns:
    -------
    Bio.Phylo.Tree
        The UPGMA tree built using Biopython.
    """
    # Convert the list-of-lists distance matrix into Biopython's DistanceMatrix format
    num_sequences = len(sequence_labels)
    matrix_data = [row[:i] for i, row in enumerate(distance_matrix, start=1)]  # Extract lower triangle (Biopython format)
    
    biopython_matrix = DistanceMatrix(names=sequence_labels, matrix=matrix_data)
    
    # Perform UPGMA clustering
    constructor = DistanceTreeConstructor()
    biopython_tree = constructor.upgma(biopython_matrix)
    
    return biopython_tree

In [ ]:
sequences = ["HER", "CAT", "CHAT"]

distance_matrix = mat_distances(sequences, blosum_m=True, gap_opening_score=-10, gap_extension_score=-2)

tree_custom = UPGMA(sequences, blosum_m=True, gap_opening_score=-10, gap_extension_score=-2)
tree_biopython = upgma_biopython(distance_matrix, sequences)

In [ ]:
print("Custom UPGMA Tree:")
visualize_tree(tree_custom, sequences)

In [ ]:
print("\nBiopython UPGMA Tree:")
draw_ascii(tree_biopython)
print(tree_biopython)

## Multiple sequence alignment

Now we will use UPGMA clustering and Needleman-Wunsch algorithm to align set of sequences.

In [ ]:
sequences = ["HAT", "HARAT", "CAT", "CHAT", "HER"]

alignment = align_multiple_sequences(sequences, True)
print()
print_alignments(alignment)

In [ ]:
sequences = ["PYNSAIRMNMQEALIVIYSYYL", "MKVPNSRMENQGALIVIDSYYLDYI", "MPDNSAIRMNME", "MKVPYNSAIRMNECFI", "MKDPYNSGIRMNMQE",
             "MKIPYNSAIRMNMQEANIYI", "AKVIYNMAIRMNMQEALIVIYSYY", "MKVPYYSAIRMNMVIWSYLQMMI", "MKVPINSAINMHMPEALIVIYSYY",
             "MNVPYAIRMNMQEALIVIYSYYLWNG", "MKVPMNSAISNMAEALIVYSCYLH", "MKNPYNSHIRMNMQEALIVIESAY", "YNSRGNMQRALIVIYSYYHKVVKL",
             "MKDEYNSAIRMMQEALIVIYSYYTLCKA", "MKVPNSAIRMNMVEALIYSYYLK", "MPVPYNSAIHSNMQE", "MKVPYNSAIMNMCEALIVIQSY", 
             "MVPPSLIRPNMQEALIVPSYLDFID", "MYQSARRMNMQEALISYYLQH", "MKVPYYSAICMNMQEA"]

alignment = align_multiple_sequences(sequences, True)
print()
print_alignments(alignment)

### Evaluation

This script compares the multiple sequence alignment (MSA) produced by our custom function `align_multiple_sequences` with the reference alignment from BALIbase.

SP-score is a widely used metric in MSA evaluation. It measures the similarity between two alignments
by counting how many columns (aligned residues) are identical in both alignments. This provides 
a fair and interpretable way to assess alignment accuracy, as structural or evolutionary relationships 
are often reflected in column-wise conservation.

In [ ]:
from Bio import AlignIO
from Bio import pairwise2
from Bio.Align import MultipleSeqAlignment
from Bio import SeqIO

In [ ]:
import os

def sp_score(alignment1, alignment2):
    """
    Computes Sum-of-Pairs (SP) score to compare two multiple sequence alignments.

    Parameters:
    ----------
    alignment1 : list of str
        The first alignment (e.g., from user's algorithm).
    alignment2 : list of str
        The reference alignment (e.g., from BALIbase).

    Returns:
    -------
    float
        The similarity score (1 = identical, 0 = completely different).
    """
    total_pairs = 0
    matching_pairs = 0

    for col in zip(*alignment1):
        total_pairs += 1
        if col in zip(*alignment2):  # Check if column exists in reference alignment
            matching_pairs += 1

    return matching_pairs / total_pairs if total_pairs > 0 else 0

In [ ]:
unaligned_file = "../balibase/RV11.unaligned/BB11001.fasta"
aligned_file = "../balibase/RV11.aligned/BB11001.fasta"

unaligned_sequences = [str(record.seq) for record in SeqIO.parse(unaligned_file, "fasta")]
reference_alignment = [str(record.seq) for record in SeqIO.parse(aligned_file, "fasta")]

aligned_sequences = align_multiple_sequences(unaligned_sequences, blosum_m=True)
print_alignments(aligned_sequences)
print(reference_alignment)

similarity = sp_score(aligned_sequences, reference_alignment)
print("SP score:", similarity)

In [ ]:
unaligned_file = "../balibase/RV11.unaligned/BB11021.fasta"
aligned_file = "../balibase/RV11.aligned/BB11021.fasta"

unaligned_sequences = [str(record.seq) for record in SeqIO.parse(unaligned_file, "fasta")]
reference_alignment = [str(record.seq) for record in SeqIO.parse(aligned_file, "fasta")]

aligned_sequences = align_multiple_sequences(unaligned_sequences, blosum_m=True)
print_alignments(aligned_sequences)
print("\nReference alignment:")
print

similarity = sp_score(aligned_sequences, reference_alignment)
print("SP score:", similarity)

In [ ]:
unaligned_file = "../balibase/RV11.unaligned/BB11030.fasta"
aligned_file = "../balibase/RV11.aligned/BB11030.fasta"

unaligned_sequences = [str(record.seq) for record in SeqIO.parse(unaligned_file, "fasta")]
reference_alignment = [str(record.seq) for record in SeqIO.parse(aligned_file, "fasta")]

aligned_sequences = align_multiple_sequences(unaligned_sequences, blosum_m=True)
print_alignments(aligned_sequences)
print("\nReference alignment:")
print

similarity = sp_score(aligned_sequences, reference_alignment)
print("SP score:", similarity)